In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_location = 'E:/Dataset/posture/'
texts = os.listdir(data_location)
texts[:3]

['cwbyoubian.txt', 'cwbyoubian2.txt', 'cwbyoubian3.txt']

### 解析文件，构造矩阵

In [4]:
#sensor1 -- sensor2 --s3 -- s4 --s5 -- s6 --s7 -- s8 --s9 -- s10 -- 名字 -- 坐姿

def text2matrix(filename):
    with open(data_location+filename) as f:
        str1 = f.readlines()
    length = len(str1)
    matrix = np.zeros((length,10))
    
    
#--------------------矩阵解析-----------------------------------------------   
    for i,line in enumerate(str1):
        if i<1:continue;#解决第一行是空行的情况
        num = np.reshape(str1[i].strip('\t\n').split('\t'),(1,-1)).shape[1] #得到行数
        if(num != 10):
            matrix[i,:num] = np.array(str1[i].strip('\t\n').split('\t'))
        else:
            matrix[i] = np.array(str1[i].strip('\t\n').split('\t'))
    handled_matrix = matrix[2:-3,:] # 去掉前2行和后3行后的矩阵
#--------------------矩阵解析-----------------------------------------------     


#--------------------label解析-----------------------------------------------  
    lab_dict = {'tuobei' :1,
                'youbian':2,
                'youpian':2,
                'zuobian':3,
                'zuopian':3,
                'zuozhi' :4}
    #根据文件名解析 特征：人 和 label
    if(filename.split('.')[0].isalpha()==False): #如果有数字
        if(filename.split('.')[0][:-1].isalpha() ==False): #进一步判断是2个数字？
            label = filename.split('.')[0][3:-2]         #如果是2个数字，就取最后两个数字之前的字母
        else:#只有一个数字
            label = filename.split('.')[0][3:-1]
    else:
        label = filename.split('.')[0][3:]
#-------------------label解析----------------------------------------------
        
#-------------------姓名解析----------------------------------------------  
    name_dict= {'cwb':1,
                'tsy':2,
                'wlj':3}
    lab_column = np.ones((handled_matrix.shape[0],1))*lab_dict[label]
    nam_column = np.ones((handled_matrix.shape[0],1))*name_dict[ filename[:3]]
    new_matrix = np.column_stack((handled_matrix,nam_column))
    matrix = np.column_stack((new_matrix,lab_column))
    
    rand_index = np.random.choice(matrix.shape[0],6,replace=False) # 抽取6个数据，四个用来训练，2个用来测试
    train_matrix = matrix[rand_index[:5]][:]
    test_matrix  = matrix[rand_index[5:]][:]
    
    return train_matrix,test_matrix

### 构造训练集和测试集

In [5]:
train_matrix = np.zeros((len(texts),5,12))
test_matrix  = np.zeros((len(texts),1,12))
def construct_dataset():
    for i,file_name in enumerate(texts):
        train_matrix[i],test_matrix[i] = text2matrix(file_name) #获得每个文件里的矩阵
    
    return train_matrix.reshape(-1,12),test_matrix.reshape(-1,12) #比较trick的一步

In [6]:
train,test = construct_dataset()
np.random.shuffle(train)
np.random.shuffle(test)
colum = ['sensor1','sensor2','sensor3','sensor4','sensor5','sensor6','sensor7','sensor8','sensor9','sensor10','Name','Posture']
x_col = ['sensor1','sensor2','sensor3','sensor4','sensor5','sensor6','sensor7','sensor8','sensor9','sensor10','Name']


In [7]:
train_df = pd.DataFrame(train,columns=colum)
test_df  = pd.DataFrame(test, columns=colum)

In [8]:
train_df.to_csv('C:/Users/ywb/Desktop/data/train_df.csv')
test_df.to_csv('C:/Users/ywb/Desktop/data/test_df.csv')

In [9]:
train_df.groupby('Name')['Posture'].count()   ## 每个人测试的数量还是平均的，不错

Name
1.0    105
2.0     90
3.0    120
Name: Posture, dtype: int64

##  特征和标签对应关系：
### 驼背：1---------- cwb ：1
### 右偏：2---------- tsy ：2
### 左偏：3---------- wlj ：3
### 坐直：4

In [10]:
train_df.Posture.value_counts()  ##有些不平衡，但是可以接受，看下训练的时候是否需要  分配权重

4.0    200
2.0     45
3.0     40
1.0     30
Name: Posture, dtype: int64

In [13]:
from sklearn.svm import SVC
from sklearn.cross_validation import KFold
from sklearn.model_selection import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import r2_score,accuracy_score,auc

In [14]:
model = SVC(decision_function_shape='ovr')
# parame = {'kernel':['rbf'], 
#           'C':np.linspace(0.1,1,num=20),
#           'gamma':np.logspace(-1,0,num=5)}
parame = {'kernel':['linear'], 
          'C':np.linspace(0.1,1,num=20)}


clf =GridSearchCV(model,parame)
clf.fit(X=train_df[x_col],y=train_df['Posture'])

clf.grid_scores_

[mean: 0.89524, std: 0.01348, params: {'C': 0.1, 'kernel': 'linear'},
 mean: 0.92063, std: 0.00849, params: {'C': 0.1473684210526316, 'kernel': 'linear'},
 mean: 0.95873, std: 0.02499, params: {'C': 0.19473684210526315, 'kernel': 'linear'},
 mean: 0.95238, std: 0.02060, params: {'C': 0.24210526315789474, 'kernel': 'linear'},
 mean: 0.95238, std: 0.02060, params: {'C': 0.2894736842105263, 'kernel': 'linear'},
 mean: 0.95238, std: 0.02060, params: {'C': 0.33684210526315794, 'kernel': 'linear'},
 mean: 0.95238, std: 0.02060, params: {'C': 0.38421052631578945, 'kernel': 'linear'},
 mean: 0.95873, std: 0.01194, params: {'C': 0.43157894736842106, 'kernel': 'linear'},
 mean: 0.96190, std: 0.00789, params: {'C': 0.4789473684210527, 'kernel': 'linear'},
 mean: 0.96190, std: 0.00789, params: {'C': 0.5263157894736842, 'kernel': 'linear'},
 mean: 0.96190, std: 0.00789, params: {'C': 0.5736842105263158, 'kernel': 'linear'},
 mean: 0.95873, std: 0.00922, params: {'C': 0.6210526315789474, 'kernel': '

In [15]:
new_model = SVC(C=0.7,kernel='rbf',gamma=0.4641)
new_model.fit(X=train_df[x_col],y=train_df['Posture'])
predict_y = new_model.predict(test_df[x_col])
print('准确率:',accuracy_score(predict_y,test_df['Posture']))

准确率: 1.0
